
# <center>Python课程实践--Gensim</center>

## 课程内容

* 1.Gensim简介
* 2.词袋模型
* 3.主题模型
* 4.词向量模型
* 5.文档向量模型
* 6.案例实践

## 1.Gensim简介

### 1.1 常用自然语言处理工具
自然语言处理（NLP）如今越来越流行，在深度学习开发的背景下变得尤为引人注目。在人工智能领域中，自然语言处理（NLP）从文本中理解和提取重要信息，并基于文本数据进行进一步的数据训练，其主要任务包括语音识别和生成、文本分析、情感分析、机器翻译等。

* NLTK（Python自然语言工具包）用于诸如标记化、词形还原、词干化、解析、POS标注等任务。该库具有几乎所有NLP任务的工具。

* Spacy是NLTK的主要竞争对手。这两个库可用于相同的任务。

* Scikit-learn为机器学习提供了一个大型库。此外还提供了用于文本预处理的工具。

* **Gensim**是一个主题和向量空间建模、文档集合相似性的工具包。

* Pattern库的一般任务是充当Web挖掘模块。因此，它仅支持自然语言处理（NLP）作为辅助任务。

* Polyglot是自然语言处理（NLP）的另一个Python工具包。它不是很受欢迎，但也可以用于各种NLP任务。

### 1.2 Gensim主要功能
作为自然语言处理爱好者，大家都应该听说过或使用过大名鼎鼎的Gensim吧，这是一款具备多种功能的神器。

Gensim是一款开源的第三方Python工具包，用于从原始的非结构化的文本中，无监督地学习到文本隐层的主题向量表达。

它支持包括TF-IDF，LSA，LDA，和word2vec在内的多种主题模型算法，支持流式训练，并提供了诸如相似度计算，信息检索等一些常用任务的API接口。

#### 基本概念

* 语料（Corpus）：一组原始文本的集合，用于无监督地训练文本主题的隐层结构。语料中不需要人工标注的附加信息。在Gensim中，Corpus通常是一个可迭代的对象（比如列表）。每一次迭代返回一个可用于表达文本对象的稀疏向量。

* 向量（Vector）：由一组文本特征构成的列表。是一段文本在Gensim中的内部表达。

* 稀疏向量（SparseVector）：通常，我们可以略去向量中多余的0元素。此时，向量中的每一个元素是一个(key, value)的元组

* 模型（Model）：是一个抽象的术语。定义了两个向量空间的变换（即从文本的一种向量表达变换为另一种向量表达）。


## 2.词袋模型

![](images/wordbag.png)

### 2.1 语料预处理
这一次，让我们从表示为字符串的文档开始：

In [2]:
from gensim import corpora
documents = ["Human machine interface for lab abc computer applications",
             "A survey of user opinion of computer system response time",
             "The EPS user interface management system",
             "System and human system engineering testing of EPS",
             "Relation of user perceived response time to error measurement",
             "The generation of random binary unordered trees",
             "The intersection graph of paths in trees",
             "Graph minors IV Widths of trees and well quasi ordering",
             "Graph minors A survey"]

这是一个由九个文档组成的小型语料库，每个文档只包含一个句子。

首先，让我们对文档进行标记，删除常用单词（使用停止词列表）以及仅在语料库中出现一次的单词：

In [3]:
# remove common words and tokenize
stoplist = set('for a of the and to in'.split()) #常用词列表
texts = [[word for word in document.lower().split() if word not in stoplist] #删除常用单词（使用停止词列表）
         for document in documents]
print(stoplist)
print(texts)

{'for', 'to', 'in', 'a', 'and', 'of', 'the'}
[['human', 'machine', 'interface', 'lab', 'abc', 'computer', 'applications'], ['survey', 'user', 'opinion', 'computer', 'system', 'response', 'time'], ['eps', 'user', 'interface', 'management', 'system'], ['system', 'human', 'system', 'engineering', 'testing', 'eps'], ['relation', 'user', 'perceived', 'response', 'time', 'error', 'measurement'], ['generation', 'random', 'binary', 'unordered', 'trees'], ['intersection', 'graph', 'paths', 'trees'], ['graph', 'minors', 'iv', 'widths', 'trees', 'well', 'quasi', 'ordering'], ['graph', 'minors', 'survey']]


In [4]:
stoplist

{'a', 'and', 'for', 'in', 'of', 'the', 'to'}

In [5]:
texts

[['human', 'machine', 'interface', 'lab', 'abc', 'computer', 'applications'],
 ['survey', 'user', 'opinion', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'management', 'system'],
 ['system', 'human', 'system', 'engineering', 'testing', 'eps'],
 ['relation', 'user', 'perceived', 'response', 'time', 'error', 'measurement'],
 ['generation', 'random', 'binary', 'unordered', 'trees'],
 ['intersection', 'graph', 'paths', 'trees'],
 ['graph', 'minors', 'iv', 'widths', 'trees', 'well', 'quasi', 'ordering'],
 ['graph', 'minors', 'survey']]

In [6]:
# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1]#删除仅在语料库中出现一次的单词
         for text in texts]
texts

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]

In [7]:
from pprint import pprint  # pretty-printer
pprint(texts)

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


### 2.2 词袋模型
要将文档转换为向量，我们将使用名为bag-of-words的文档表示 。在此表示中，每个文档由一个向量表示，其中每个向量元素表示问题 - 答案对，仅通过它们的（整数）id来表示问题是有利的。问题和ID之间的映射称为字典：

In [8]:
dictionary = corpora.Dictionary(texts) #创建一个映射字典
dictionary.save('tmp/deerwester.dict')  # store the dictionary, for future reference
print(dictionary)

C:\ProgramData\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...)


In [9]:
dictionary

在这里，我们为语料库中出现的所有单词分配了一个唯一的整数id gensim.corpora.dictionary.Dictionary。这会扫描文本，收集字数和相关统计数据。最后，我们看到在处理过的语料库中有12个不同的单词，这意味着每个文档将由12个数字表示（即，通过12-D向量）。要查看单词及其ID之间的映射：

In [10]:
print(dictionary.token2id)

{'computer': 0, 'human': 1, 'interface': 2, 'response': 3, 'survey': 4, 'system': 5, 'time': 6, 'user': 7, 'eps': 8, 'trees': 9, 'graph': 10, 'minors': 11}


要将标记化文档实际转换为向量：

In [11]:
new_doc = "Human computer interaction"
new_vec = dictionary.doc2bow(new_doc.lower().split())  #使用我们刚构造的字典来进行编码
print(new_vec)

[(0, 1), (1, 1)]


该函数doc2bow()只计算每个不同单词的出现次数，将单词转换为整数单词id，并将结果作为稀疏向量返回。 因此，稀疏向量 [(0, 1), (1, 1)] 读取：在文档“人机交互”中，单词computer （id 0）和human（id 1）出现一次; 其他十个字典单词（隐含地）出现零次。

In [12]:
print(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('tmp/deerwester.mm', corpus)  # store to disk, for later use
print(corpus)

[['human', 'interface', 'computer'], ['survey', 'user', 'computer', 'system', 'response', 'time'], ['eps', 'user', 'interface', 'system'], ['system', 'human', 'system', 'eps'], ['user', 'response', 'time'], ['trees'], ['graph', 'trees'], ['graph', 'minors', 'trees'], ['graph', 'minors', 'survey']]
[[(0, 1), (1, 1), (2, 1)], [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)], [(2, 1), (5, 1), (7, 1), (8, 1)], [(1, 1), (5, 2), (8, 1)], [(3, 1), (6, 1), (7, 1)], [(9, 1)], [(9, 1), (10, 1)], [(9, 1), (10, 1), (11, 1)], [(4, 1), (10, 1), (11, 1)]]


## 3.主题模型

对文本向量的变换是Gensim的核心。通过挖掘语料中隐藏的语义结构特征，我们最终可以变换出一个简洁高效的文本向量。

### 3.1 TF-IDF模型

在Gensim中，每一个向量变换的操作都对应着一个主题模型，例如上一小节提到的对应着词袋模型的doc2bow变换。每一个模型又都是一个标准的Python对象。下面以TF-IDF模型为例，介绍Gensim模型的一般使用方法。

首先是模型对象的初始化。通常，Gensim模型都接受一段训练语料（注意在Gensim中，语料对应着一个稀疏向量的迭代器）作为初始化的参数。显然，越复杂的模型需要配置的参数越多。

In [13]:
# 创建转换
from gensim import models
tfidf = models.TfidfModel(corpus)

其中，corpus是一个返回bow向量的迭代器。这两行代码将完成对corpus中出现的每一个特征的IDF值的统计工作。

接下来，我们可以调用这个模型将任意一段语料（依然是bow向量的迭代器）转化成TFIDF向量（的迭代器）。需要注意的是，这里的bow向量必须与训练语料的bow向量共享同一个特征字典（即共享同一个向量空间）。


TF-IDF用以评估一字词对于一个文件集或一个语料库中的其中一份文件的重要程度。字词的重要性随着它在文件中出现的次数成正比增加，但同时会随着它在语料库中出现的频率成反比下降。

TF:词频，指的是某一个给定的词语在该文件中出现的频率

IDF:倒文档词频，是一个词语普遍重要性的度量，可以由总文件数目除以包含该词语之文件的数目，再将得到的商取对数得到。

挑选文档的特征词：某一特定文件内的高词语频率，以及该词语在整个文件集合中的低文件频率，可以产生出高权重的TF-IDF。

TF-IDE=TF*IDF


In [14]:
# 变换向量
doc_bow = [(0, 1), (1, 1)]
print(tfidf[doc_bow]) # step 2 -- use the model to transform vectors
#输出每个词 在这个文档中重要性

[(0, 0.7071067811865476), (1, 0.7071067811865476)]


In [15]:
# 或者将转换应用于整个语料库：
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
    print(doc)

[(0, 0.5773502691896257), (1, 0.5773502691896257), (2, 0.5773502691896257)]
[(0, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.44424552527467476), (5, 0.3244870206138555), (6, 0.44424552527467476), (7, 0.3244870206138555)]
[(2, 0.5710059809418182), (5, 0.4170757362022777), (7, 0.4170757362022777), (8, 0.5710059809418182)]
[(1, 0.49182558987264147), (5, 0.7184811607083769), (8, 0.49182558987264147)]
[(3, 0.6282580468670046), (6, 0.6282580468670046), (7, 0.45889394536615247)]
[(9, 1.0)]
[(9, 0.7071067811865475), (10, 0.7071067811865475)]
[(9, 0.5080429008916749), (10, 0.5080429008916749), (11, 0.695546419520037)]
[(4, 0.6282580468670046), (10, 0.45889394536615247), (11, 0.6282580468670046)]


In [16]:
# 可以将训练好的模型持久化到磁盘上，以便下一次使用：
tfidf.save("tmp/model.tfidf")
new_tfidf = models.TfidfModel.load("tmp/model.tfidf")

C:\ProgramData\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


### 3.2 LSA模型

词袋模型有几个明显的缺点：

* 稀疏性(Sparseness)：对于大词袋，尤其是包含了生僻字的词袋，文档稀疏性不可避免。
* 一词多义：一词多义是一种常见的现象，但是BOW模型只统计单词出现的次数，而忽略了他们之间的区别；
* 多词一义：在不同或相同文档中，可能存在多种单词存在同一个意思。

为了解决同义词以及文档稀疏性的问题，在BOW模型中引入“主题”的概念，提出了潜在语义分析LSA(Latent semantic analysis)模型，，其通过“矢量语义空间”来提取文档与词中的“概念”，进而分析文档与词之间的关系。LSA的基本假设是，如果两个词多次出现在同一文档中，则这两个词在语义上具有相似性。LSA使用大量的文本上构建一个矩阵，这个矩阵的一行代表一个词，一列代表一个文档，矩阵元素代表该词在该文档中出现的次数，然后再此矩阵上使用奇异值分解（SVD）来保留列信息的情况下减少矩阵行数，之后每两个词语的相似性则可以通过其行向量的cos值（或者归一化之后使用向量点乘）来进行标示，此值越接近于1则说明两个词语越相似，越接近于0则说明越不相似。

In [17]:
lsi = models.LsiModel(corpus=corpus, id2word=dictionary, num_topics=5)
lsi

In [18]:
# 变换向量
doc_bow = [(0, 1), (1, 1)]
print(lsi[doc_bow]) # step 2 -- use the model to transform vectors
#输出每个词 在这个文档中重要性

[(0, 0.4618210045327158), (1, 0.07002766527899965), (2, -0.12452907551899137), (3, 1.0097125584438555), (4, -0.2130304060562643)]


In [19]:
# 或者将转换应用于整个语料库：
corpus_lsi = lsi[corpus]
for doc in corpus_lsi:
    print(doc)

[(0, 0.6594664059797395), (1, 0.14211544403729823), (2, -0.259568714208422), (3, 1.561952142099365), (4, 0.06873853289228224)]
[(0, 2.024543043382876), (1, -0.4208875824630211), (2, 1.1706784758367), (3, 0.04711403946278517), (4, -0.3104877971509376)]
[(0, 1.5465535813286544), (1, 0.3235891942571181), (2, -0.4886926871430969), (3, -0.06842766506520662), (4, 0.5693322998862521)]
[(0, 1.8111412473028827), (1, 0.589052496993243), (2, -1.3415628918291795), (3, -0.44026438019664316), (4, -0.3094004567039055)]
[(0, 0.9336738035634353), (1, -0.2713894049937495), (2, 1.189800531276366), (3, -0.24673811387727104), (4, 0.4923724704004483)]
[(0, 0.012746183038294471), (1, -0.490161792453105), (2, -0.231120154886051), (3, -0.02480199852746337), (4, 0.5941695155888926)]
[(0, 0.04888203206047028), (1, -1.112947026992956), (2, -0.45420651747622043), (3, -0.025502070648909945), (4, 0.5259165773893245)]
[(0, 0.08063836099410607), (1, -1.5634559463442665), (2, -0.5953216813655241), (3, -0.01677260003833

In [20]:
# 可以将训练好的模型持久化到磁盘上，以便下一次使用：
lsi.save("tmp/model.lsi")
new_lsi = models.LsiModel.load("tmp/model.lsi")

C:\ProgramData\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
C:\ProgramData\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
C:\ProgramData\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-fu

### 3.3 LDA模型
人类是怎么生成文档的呢？

比如假设事先给定了这几个主题：Arts、Budgets、Children、Education，然后通过学习训练，获取每个主题Topic对应的词语。如下图所示：
![](images/lda2.jpg)

然后以一定的概率选取上述某个主题，再以一定的概率选取那个主题下的某个单词，不断的重复这两步，最终生成如下图所示的一篇文章（其中不同颜色的词语分别对应上图中不同主题下的词）
![](images/lda1.jpg)

In [64]:
lda = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=20)
print(corpus)

MmCorpus(9 documents, 12 features, 28 non-zero entries)


In [65]:
print(dictionary)

Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...)


In [22]:
# 变换向量
doc_bow = [(0, 1), (1, 1)]
print(lda[doc_bow]) # step 2 -- use the model to transform vectors
#输出每个词 在这个文档中重要性

[(0, 0.016666677), (1, 0.016666677), (2, 0.016666677), (3, 0.016666677), (4, 0.016666677), (5, 0.016666677), (6, 0.016666677), (7, 0.016666677), (8, 0.016666677), (9, 0.016666677), (10, 0.016666677), (11, 0.016666677), (12, 0.6833331), (13, 0.016666677), (14, 0.016666677), (15, 0.016666677), (16, 0.016666677), (17, 0.016666677), (18, 0.016666677), (19, 0.016666677)]


In [23]:
# 或者将转换应用于整个语料库：
corpus_lda = lda[corpus]
for doc in corpus_lda:
    print(doc)

[(0, 0.012500009), (1, 0.012500009), (2, 0.012500009), (3, 0.012500009), (4, 0.012500009), (5, 0.012500009), (6, 0.012500009), (7, 0.012500009), (8, 0.012500009), (9, 0.012500009), (10, 0.012500009), (11, 0.012500009), (12, 0.7624998), (13, 0.012500009), (14, 0.012500009), (15, 0.012500009), (16, 0.012500009), (17, 0.012500009), (18, 0.012500009), (19, 0.012500009)]
[(15, 0.8642855)]
[(0, 0.010000009), (1, 0.010000009), (2, 0.010000009), (3, 0.010000009), (4, 0.010000009), (5, 0.010000009), (6, 0.010000009), (7, 0.010000009), (8, 0.010000009), (9, 0.010000009), (10, 0.010000009), (11, 0.010000009), (12, 0.010000009), (13, 0.010000009), (14, 0.010000009), (15, 0.010000009), (16, 0.010000009), (17, 0.8099999), (18, 0.010000009), (19, 0.010000009)]
[(0, 0.010000013), (1, 0.010000013), (2, 0.010000013), (3, 0.010000013), (4, 0.010000013), (5, 0.010000013), (6, 0.010000013), (7, 0.010000013), (8, 0.010000013), (9, 0.010000013), (10, 0.010000013), (11, 0.010000013), (12, 0.010000013), (13, 0

In [24]:
# 可以将训练好的模型持久化到磁盘上，以便下一次使用：
lda.save("tmp/model.lda")
new_lda = models.LdaModel.load("tmp/model.lda")

C:\ProgramData\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
C:\ProgramData\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
C:\ProgramData\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-fu

### 3.4 文档相似度的计算
在得到每一篇文档对应的主题向量后，我们就可以计算文档之间的相似度，进而完成如文本聚类、信息检索之类的任务。在Gensim中，也提供了这一类任务的API接口。

以信息检索为例。对于一篇待检索的query，我们的目标是从文本集合中检索出主题相似度最高的文档。

首先，我们需要将待检索的query和文本放在同一个向量空间里进行表达（以LSI向量空间为例）：

In [25]:
from gensim import corpora, models, similarities
dictionary = corpora.Dictionary.load('tmp/deerwester.dict')
corpus = corpora.MmCorpus('tmp/deerwester.mm') # comes from the first tutorial, "From strings to vectors"
print(corpus)

MmCorpus(9 documents, 12 features, 28 non-zero entries)


In [26]:
# 定义一个二维LSI空间：
new_lsi = models.LsiModel.load("tmp/model.lsi")

C:\ProgramData\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [27]:
doc = "Human computer interaction"
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = new_lsi[vec_bow] # convert the query to LSI space
print(vec_lsi)

[(0, 0.4618210045327158), (1, 0.07002766527899965), (2, -0.12452907551899137), (3, 1.0097125584438555), (4, -0.2130304060562643)]


In [28]:
# 初始化查询
index = similarities.MatrixSimilarity(new_lsi[corpus])

* similarities.MatrixSimilarity只有当整个向量集适合内存时，该类才适用。例如，当与此类一起使用时，一百万个文档的语料库在256维LSI空间中将需要2GB的RAM。
* 如果没有2GB的可用RAM，则需要使用similarities.Similarity该类。此类通过在磁盘上的多个文件（称为分片）之间拆分索引，在固定内存中运行。它使用similarities.MatrixSimilarity和similarities.SparseMatrixSimilarity内部，所以它仍然很快，虽然稍微复杂一点。

In [29]:
# 要获得我们的查询文档与九个索引文档的相似性：
sims = index[vec_lsi] # perform a similarity query against the corpus
print(list(enumerate(sims))) # print (document_number, document_similarity) 2-tuples

#余弦测量返回范围中的相似度（越大，越相似）

[(0, 0.9727242), (1, 0.31977737), (2, 0.30448094), (3, 0.24447846), (4, -0.048387423), (5, -0.16497497), (6, -0.09127022), (7, -0.03274583), (8, 0.15309833)]


In [30]:
# 将这些相似性按降序排序，并获得查询 人机交互 的最终答案：
sims = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims) # print sorted (document number, similarity score) 2-tuples

[(0, 0.9727242), (1, 0.31977737), (2, 0.30448094), (3, 0.24447846), (8, 0.15309833), (7, -0.03274583), (4, -0.048387423), (6, -0.09127022), (5, -0.16497497)]


## 4.词向量模型

### 4.1 词向量简介

词向量指的是一个词的向量表示。如果你希望计算机能够进行一些复杂点的文本语义学习，你必须得将文本数据编码成计算机能够处理的数值向量吧，所以词向量是一个自然语言处理任务中非常重要的一环。

如何得到上述具有语义Distributional  representations的词向量呢，2013年提出的word2vec的方法就是一种非常方便得到高质量词向量的方式。其主要思想是：一个词的上下文可以很好的表达出词的语义，它是一种通过无监督的学习文本来用产生词向量的方式。

100+ Chinese Word Vectors 上百种预训练中文词向量
https://github.com/Embedding/Chinese-Word-Vectors

In [31]:
from gensim import models
new_model = models.KeyedVectors.load_word2vec_format('tmp/zhwiki_2017_03.sg_50d.word2vec',binary=False)  #sgns.wiki.bigram-char

In [32]:
print(new_model['软件'])

[-0.36093   0.105888  0.388663 -0.117126  0.106318  0.350478 -0.081701
 -0.059811  0.847456  0.848872  0.607633 -1.150333  0.781525  0.037457
  0.047236 -0.458902  0.412925  0.163321 -0.007959  0.23333  -0.416148
 -0.441578 -0.828599 -0.003355  0.536412 -0.288777 -0.007912 -0.116614
  0.749254 -0.453145 -0.368196  0.273862 -0.859477  0.776925 -0.229281
  0.532504 -0.775858  0.08166  -0.160816 -0.441611  0.156932  0.493826
  0.572834 -0.218519 -0.174441 -0.153281 -0.239637 -0.185816 -0.013873
 -0.146506]


In [33]:
# 得到与一个词最相关的若干词及相似程度
print(new_model.similar_by_word('北京'), 10)

[('上海', 0.85248863697052), ('广州', 0.8133988380432129), ('南京', 0.8087798953056335), ('沈阳', 0.8043987154960632), ('长春', 0.7917946577072144), ('哈尔滨', 0.7850497364997864), ('杭州', 0.7794261574745178), ('济南', 0.7794216871261597), ('福寿园', 0.7779054045677185), ('潞河', 0.7772673964500427)] 10


In [34]:
print(new_model.similar_by_word('王敬'), 10)

[('张亮', 0.8167524337768555), ('郭丹', 0.800384521484375), ('颜高', 0.7938944101333618), ('朱俊', 0.7935847043991089), ('王成', 0.7926192879676819), ('瑕', 0.7872204184532166), ('李云', 0.7860795855522156), ('王亮', 0.7809092998504639), ('张雕', 0.7777642011642456), ('又击', 0.7747111320495605)] 10


In [35]:
new_model.similar_by_word('Python')

[('Tcl', 0.9650475978851318),
 ('Java', 0.9643802642822266),
 ('Perl', 0.9631242752075195),
 ('Lua', 0.9580066800117493),
 ('C#', 0.9556556940078735),
 ('java', 0.9553079605102539),
 ('脚本语言', 0.9521129131317139),
 ('C语言', 0.9500064849853516),
 ('程式库', 0.947653591632843),
 ('JavaScript', 0.9476410150527954)]

In [36]:
# 得到两组词的相似度
list1 = [u'核能']
list2 = [u'电能']
list3 = [u'电力']
list_sim1 =  new_model.n_similarity(list1, list2)
print(list_sim1)
list_sim2 = new_model.n_similarity(list2, list3)
print(list_sim2)

0.59094995
0.83814484


In [37]:
y2 = new_model.wv.similarity(u"时期", u"种类")
print(y2)

0.23437281


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [38]:
for i in new_model.wv.most_similar(u"戏剧"):
    print (i[0],i[1])

话剧 0.8621609807014465
舞台剧 0.8580252528190613
表演艺术 0.8423406481742859
舞台艺术 0.8419910073280334
戏曲 0.8403540849685669
音乐剧 0.8395066857337952
编导 0.8327195644378662
喜剧 0.8309129476547241
儿童剧 0.8290919065475464
剧作 0.8268210291862488


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [39]:
# 得到一组词中最无关的词
list4 = [u'汽车', u'火车', u'飞机', u'北京']
print(new_model.doesnt_match(list4))

北京


C:\ProgramData\Anaconda3\lib\site-packages\gensim\models\keyedvectors.py:858: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


#### 词语的加减游戏
英文词类比中最有名的一个例子大概就是: king - man + woman = queen

当我把这个例子换成中文映射到腾讯的中文词向量中并且用gensim来计算，竟然能完美复现：国王 - 男人 + 女人 = 王后

In [40]:
new_model.most_similar(positive=[u'国王', '女人'], negative=[u'男人'])

[('三世', 0.9416244029998779),
 ('一世', 0.9366082549095154),
 ('二世', 0.9362475872039795),
 ('四世', 0.9269468784332275),
 ('五世', 0.9181405305862427),
 ('六世', 0.9168587923049927),
 ('王位', 0.914807915687561),
 ('王室', 0.9138323068618774),
 ('七世', 0.9131208658218384),
 ('王路易', 0.9111517667770386)]

In [41]:
new_model.most_similar(positive=[u'皇帝', '女人'], negative=[u'男人'])

[('登基', 0.8970834016799927),
 ('王太子', 0.8666362762451172),
 ('储君', 0.8532828688621521),
 ('新君', 0.8518441319465637),
 ('尊号', 0.8511227369308472),
 ('继位', 0.8454713821411133),
 ('废黜', 0.845383882522583),
 ('即位', 0.8407121896743774),
 ('帝号', 0.839134693145752),
 ('死后', 0.8382000923156738)]

In [42]:
new_model.most_similar(positive=[u'老婆', '妻子'], negative=[u'老公'])

[('女儿', 0.916805624961853),
 ('丈夫', 0.9024686813354492),
 ('母亲', 0.9016439914703369),
 ('父亲', 0.8894546627998352),
 ('儿子', 0.8886772394180298),
 ('哥哥', 0.886040449142456),
 ('小儿子', 0.8813856840133667),
 ('叔叔', 0.8799700736999512),
 ('小女儿', 0.8782185316085815),
 ('舅舅', 0.8779697418212891)]

然后把国王换成皇帝，将输出什么？
```
机场-飞机+火车
老婆-老公+丈夫
北京-中国+法国
上海-中国+美国
小龙女-杨过+郭靖
汽车-轮胎+翅膀
```

### 4.2 训练词向量

word2vec中有两个非常经典的模型：skip-gram和cbow。

* skip-gram：已知中心词预测周围词。
* cbow：已知周围词预测中心词。

比如 "the quick brown fox jumps over the lazy dog" 如果定义window-size为2的话，就会产生如下图所示的数据集，window-size决定了目标词会与多远距离的上下文产生关系：

```
Skip-Gram:(the,quick) ,其中the 是模型的输入，quick是模型的输出。
Cbow： ((quick,brown),the) ,其中 （quick，brown）是模型的输入，the是模型的输出。
```
![](images/word2vec.png)

使用word2vec.Word2Vec这个API的来训练词向量，参数说明如下
```
size: 表示词向量的维度，默认值是100。
window：决定了目标词会与多远距离的上下文产生关系，默认值是5。
sg: 如果是0， 则是CBOW模型，是1则是Skip-Gram模型，默认是0即CBOW模型。
```
采用CBOW模型——通过周围词预测中心词的方式训练词向量。

In [43]:
import os
fpath = 'data/IT'
flist = os.listdir(fpath)
flist

['IT_10.txt',
 'IT_100.txt',
 'IT_1000.txt',
 'IT_1010.txt',
 'IT_1020.txt',
 'IT_1030.txt',
 'IT_1040.txt',
 'IT_1050.txt',
 'IT_1060.txt',
 'IT_1070.txt',
 'IT_1080.txt',
 'IT_1090.txt',
 'IT_110.txt',
 'IT_1100.txt',
 'IT_1110.txt',
 'IT_1120.txt',
 'IT_1130.txt',
 'IT_1140.txt',
 'IT_1150.txt',
 'IT_1160.txt',
 'IT_1170.txt',
 'IT_1180.txt',
 'IT_1190.txt',
 'IT_120.txt',
 'IT_1200.txt',
 'IT_1210.txt',
 'IT_1220.txt',
 'IT_1230.txt',
 'IT_1240.txt',
 'IT_1250.txt',
 'IT_1260.txt',
 'IT_1270.txt',
 'IT_1280.txt',
 'IT_1290.txt',
 'IT_130.txt',
 'IT_1300.txt',
 'IT_1310.txt',
 'IT_1320.txt',
 'IT_1330.txt',
 'IT_1340.txt',
 'IT_1350.txt',
 'IT_1360.txt',
 'IT_1370.txt',
 'IT_1380.txt',
 'IT_1390.txt',
 'IT_140.txt',
 'IT_1400.txt',
 'IT_1410.txt',
 'IT_1420.txt',
 'IT_1430.txt',
 'IT_1440.txt',
 'IT_1450.txt',
 'IT_1460.txt',
 'IT_1470.txt',
 'IT_1480.txt',
 'IT_1490.txt',
 'IT_150.txt',
 'IT_1500.txt',
 'IT_1510.txt',
 'IT_1520.txt',
 'IT_1530.txt',
 'IT_1540.txt',
 'IT_1550.txt',


In [44]:
datas = list()
for i in range(0, len(flist)):
    path = os.path.join(fpath, flist[i])
    with open(path, 'r', encoding='utf-8') as f:
        datas.append(f.read().strip())
datas

['本报讯 记者 王京 联想 THINKPAD 近期 几乎 系列 笔记本 电脑 降价促销 最高 降幅 达到 降幅 达到 42% 记者 联想 美国 官方 网站 发现 联想 相关 人士 表示 纪念 联想 成立 美国 市场 推出 促销 产品 包括 THINKPAD T 系列 笔记本 促销 价格战 THINK 品牌 高端 商务 路线 方向 不会 改变',
 '赛迪网 消息 思科 周二 发布 其财 第三季度 财报 财报 显示 股票 期权 开支 收购 相关 费用 影响 季度 营收 实现 增长 净利润 同比 下降 \u3000据 Maketwatch 网站 报道 思科 季度 实现 净利润 低于 去年 同期 本季度 收益 去年 同期 收益 思科 去年 同期 发行 股票 数量 于今 Thomson First Call 调查 分析师 预计 收益 \u3000本 季度 营收 同比 增长 高于 华尔街 分析师 预计 思科 新近 收购 Scientific-Atlanta 公司 季度 营收 增长 做出 贡献 思科 上个世纪 年代 进行 收购 实现 发展 目的 现在 希望 借助 价格 收购 Scientific-Atlanta 公司 扩大 互联网 设备 市场 份额 分析师 投资者 看好 思科 Scientific-Atlanta 收购 思科 股价 进入 今年 上涨 财报 发布 思科 股票 盘后 交易 上涨',
 'eNet 硅谷动力 消息 国外 媒体 报道 一名 德国 数据库安全 工程师 日前 撰文 指出 目前 甲骨文 软件 产品 尚未 修补 漏洞 漏洞 发现 不等 德国 信息 安全 机构 红色 数据库安全 工程师 亚力山大 ·科布鲁斯特 日前 Buqtraq 安全 邮件 列表 发表 文章 漏洞 集中 甲骨文 数据库 产品 漏洞 之前 发现 报告 甲骨文 最新 漏洞 之前 发现 \u3000科 布鲁斯特 谈及 漏洞 包括 SQL 语句 插入 漏洞 交叉 脚本 攻击 漏洞 明文 密码 泄露 漏洞 漏洞 甲骨文公司 未来 安全更新 中将 得到 修补 清楚 甲骨文 升级 软件 发布 修补 具体 漏洞 时间 去年 甲骨文 首席 安全 玛丽·安娜·戴维德森 席话 信息 安全界 引发 骚动 安娜 已经 发现 甲骨文 软件 漏洞 有四 公司 工程师 发现 科布鲁斯特 计算 甲骨文 软件 尚未 得到 修补 

In [45]:
sentences = list()
for data in datas:
    sentences.append(data.split())
sentences

[['本报讯',
  '记者',
  '王京',
  '联想',
  'THINKPAD',
  '近期',
  '几乎',
  '系列',
  '笔记本',
  '电脑',
  '降价促销',
  '最高',
  '降幅',
  '达到',
  '降幅',
  '达到',
  '42%',
  '记者',
  '联想',
  '美国',
  '官方',
  '网站',
  '发现',
  '联想',
  '相关',
  '人士',
  '表示',
  '纪念',
  '联想',
  '成立',
  '美国',
  '市场',
  '推出',
  '促销',
  '产品',
  '包括',
  'THINKPAD',
  'T',
  '系列',
  '笔记本',
  '促销',
  '价格战',
  'THINK',
  '品牌',
  '高端',
  '商务',
  '路线',
  '方向',
  '不会',
  '改变'],
 ['赛迪网',
  '消息',
  '思科',
  '周二',
  '发布',
  '其财',
  '第三季度',
  '财报',
  '财报',
  '显示',
  '股票',
  '期权',
  '开支',
  '收购',
  '相关',
  '费用',
  '影响',
  '季度',
  '营收',
  '实现',
  '增长',
  '净利润',
  '同比',
  '下降',
  '据',
  'Maketwatch',
  '网站',
  '报道',
  '思科',
  '季度',
  '实现',
  '净利润',
  '低于',
  '去年',
  '同期',
  '本季度',
  '收益',
  '去年',
  '同期',
  '收益',
  '思科',
  '去年',
  '同期',
  '发行',
  '股票',
  '数量',
  '于今',
  'Thomson',
  'First',
  'Call',
  '调查',
  '分析师',
  '预计',
  '收益',
  '本',
  '季度',
  '营收',
  '同比',
  '增长',
  '高于',
  '华尔街',
  '分析师',
  '预计',
  '思科',
  '新近',
  '收购',
  'Scientific-Atlanta',
 

In [46]:
from gensim.models import Word2Vec
model = Word2Vec(sentences, size=100, window=5, min_count=5, workers=4)
model.save('my.word2vec')

C:\ProgramData\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [47]:
model = Word2Vec.load('my.word2vec')

C:\ProgramData\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [48]:
model.wv['电脑']

array([ 0.18344449,  0.0152768 , -0.13604826, -0.01214636,  0.08696793,
       -0.2043133 , -0.08343486, -0.20476754, -0.30961055, -0.05255929,
        0.26758552, -0.09658953, -0.49444154,  0.5289667 ,  0.15155792,
       -0.2046119 , -0.05785393,  0.36078644,  0.09493233, -0.06579098,
       -0.42164695,  0.0755384 ,  0.32462794, -0.0837433 ,  0.03612296,
        0.18483846,  0.26904422,  0.13077937, -0.26698858,  0.40861133,
       -0.08559329,  0.27656752, -0.38725853, -0.24136145, -0.09527284,
       -0.6075816 , -0.12395583, -0.41609052, -0.54861623, -0.21724907,
        0.1370065 ,  0.5673256 ,  0.1917161 ,  0.1599618 , -0.6628393 ,
        0.04466977, -0.15765296, -0.30242646, -0.25842968, -0.25783616,
        0.03822951, -0.1252077 ,  0.4490651 , -0.01014764, -0.01515726,
        0.32964355, -0.83499193,  0.6825774 , -0.10666229,  0.22176291,
        0.17136481, -0.12737551,  0.07468461,  0.02523253,  0.06961744,
        0.3637891 ,  0.02655508, -0.06998077, -0.10937472,  0.01

In [49]:
model.similarity(u'笔记本', u'电脑')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.9998558

#### 词向量增量训练

```
model = gensim.models.Word2Vec.load(fname)
model.train(sentences,total_examples=new_model.corpus_count, epochs=1)
```

#### 数据降维与可视化

降维就是用2维或3维表示多维数据（彼此具有相关性的多个特征数据）的技术，利用降维算法，可以显式地表现数据。（t-SNE）t分布随机邻域嵌入 是一种用于探索高维数据的非线性降维算法。它将多维数据映射到适合于人类观察的两个或多个维度。

In [50]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

wordVocab = [k for (k,v) in model.wv.vocab.items()]

random_word = wordVocab[50:100]
X_tsne = TSNE(n_components=2, learning_rate=100).fit_transform(model.wv[random_word])

plt.figure(figsize=(14, 8))
plt.scatter(X_tsne[:, 0], X_tsne[:, 1])
for i in range(len(X_tsne)):
    x = X_tsne[i][0]
    y = X_tsne[i][1]
    plt.text(x, y, random_word[i], size=16)
plt.show()

<Figure size 1400x800 with 1 Axes>

## 5.文档向量模型

### 5.1 Doc2vec原理

前文总结了Word2vec训练词向量的细节，讲解了一个词是如何通过word2vec模型训练出唯一的向量来表示的。那接着可能就会想到，有没有什么办法能够将一个句子甚至一篇短文也用一个向量来表示呢？答案是肯定有的，构建一个句子向量有很多种方法，今天我们接着word2vec来介绍下Doc2vec，看下Doc2vec是怎么训练一个句子向量的。

Doc2vec又叫Paragraph Vector是Tomas Mikolov基于word2vec模型提出的，其具有一些优点，比如不用固定句子长度，接受不同长度的句子做训练样本，Doc2vec是一个无监督学习算法，该算法用于预测一个向量来表示不同的文档，该模型的结构潜在的克服了词袋模型的缺点。

和word2vec一样，Doc2vec也有两种训练方式，一种是PV-DM（Distributed Memory Model of paragraphvectors）类似于word2vec中的CBOW模型，如图一：

![](images/doc1.jpeg)

另一种是PV-DBOW（Distributed Bag of Words of paragraph vector)类似于word2vec中的skip-gram模型，如图二：

![](images/doc2.jpeg)

在Doc2vec中，每一句话用唯一的向量来表示，用矩阵D的某一列来代表。每一个词也用唯一的向量来表示，用矩阵W的某一列来表示。每次从一句话中滑动采样固定长度的词，取其中一个词作预测词，其他的作输入词。输入词对应的词向量word vector和本句话对应的句子向量Paragraph vector作为输入层的输入，将本句话的向量和本次采样的词向量相加求平均或者累加构成一个新的向量X，进而使用这个向量X预测此次窗口内的预测词。

In [51]:
sentences

[['本报讯',
  '记者',
  '王京',
  '联想',
  'THINKPAD',
  '近期',
  '几乎',
  '系列',
  '笔记本',
  '电脑',
  '降价促销',
  '最高',
  '降幅',
  '达到',
  '降幅',
  '达到',
  '42%',
  '记者',
  '联想',
  '美国',
  '官方',
  '网站',
  '发现',
  '联想',
  '相关',
  '人士',
  '表示',
  '纪念',
  '联想',
  '成立',
  '美国',
  '市场',
  '推出',
  '促销',
  '产品',
  '包括',
  'THINKPAD',
  'T',
  '系列',
  '笔记本',
  '促销',
  '价格战',
  'THINK',
  '品牌',
  '高端',
  '商务',
  '路线',
  '方向',
  '不会',
  '改变'],
 ['赛迪网',
  '消息',
  '思科',
  '周二',
  '发布',
  '其财',
  '第三季度',
  '财报',
  '财报',
  '显示',
  '股票',
  '期权',
  '开支',
  '收购',
  '相关',
  '费用',
  '影响',
  '季度',
  '营收',
  '实现',
  '增长',
  '净利润',
  '同比',
  '下降',
  '据',
  'Maketwatch',
  '网站',
  '报道',
  '思科',
  '季度',
  '实现',
  '净利润',
  '低于',
  '去年',
  '同期',
  '本季度',
  '收益',
  '去年',
  '同期',
  '收益',
  '思科',
  '去年',
  '同期',
  '发行',
  '股票',
  '数量',
  '于今',
  'Thomson',
  'First',
  'Call',
  '调查',
  '分析师',
  '预计',
  '收益',
  '本',
  '季度',
  '营收',
  '同比',
  '增长',
  '高于',
  '华尔街',
  '分析师',
  '预计',
  '思科',
  '新近',
  '收购',
  'Scientific-Atlanta',
 

In [52]:
sentences[0]

['本报讯',
 '记者',
 '王京',
 '联想',
 'THINKPAD',
 '近期',
 '几乎',
 '系列',
 '笔记本',
 '电脑',
 '降价促销',
 '最高',
 '降幅',
 '达到',
 '降幅',
 '达到',
 '42%',
 '记者',
 '联想',
 '美国',
 '官方',
 '网站',
 '发现',
 '联想',
 '相关',
 '人士',
 '表示',
 '纪念',
 '联想',
 '成立',
 '美国',
 '市场',
 '推出',
 '促销',
 '产品',
 '包括',
 'THINKPAD',
 'T',
 '系列',
 '笔记本',
 '促销',
 '价格战',
 'THINK',
 '品牌',
 '高端',
 '商务',
 '路线',
 '方向',
 '不会',
 '改变']

In [53]:
import gensim
# Create the tagged document needed for Doc2Vec
for i in range(len(sentences)):
    sentences[i] = gensim.models.doc2vec.TaggedDocument(words = sentences[i], tags = [i]) 
train_data=sentences

In [54]:
type(sentences)

list

模型训练
```py
model = Doc2Vec(documents = sentences, dm = 1, size = 100, window = 3, min_count = 1, iter = 10, workers = Pool()._processes)
```
参数说明
* documents: training data (has to be iterable TaggedDocument instances)
* size: 向量的维度
* dm: 1 PV-DM, 0 PV-DBOW
* window: 上下文词语离当前词语的最大距离
* min_count: 词频小于min_count的词会被忽略
* iter: 在整个语料上的迭代次数(epochs)，推荐10到20
* workers: number of worker threads to train

In [55]:
# Init the Doc2Vec model
model = gensim.models.doc2vec.Doc2Vec(dm=1, size=100, window=5, min_count=2, workers=4)

# Build the Volabulary
model.build_vocab(sentences)

# Train the Doc2Vec model
model.train(sentences, total_examples=model.corpus_count, epochs=5)

C:\ProgramData\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:580: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [56]:
# 与标签‘0’最相似的
model.docvecs.most_similar(0)

[(123, 0.9938210248947144),
 (198, 0.9937419891357422),
 (34, 0.9937158823013306),
 (142, 0.9935526847839355),
 (28, 0.9935396313667297),
 (73, 0.9935389757156372),
 (118, 0.9934990406036377),
 (13, 0.9934804439544678),
 (196, 0.9934653043746948),
 (112, 0.9934574961662292)]

In [57]:
# 进行相关性比较
model.docvecs.similarity(0,2)

0.99245334

In [58]:
# 输出标签为‘0’句子的向量
model.docvecs[0]

array([ 8.47692415e-03, -5.72356349e-03, -1.33544700e-02, -9.71434265e-03,
       -5.76193165e-03,  1.55212674e-02,  1.67094283e-02, -7.87180557e-04,
       -1.75144132e-02, -2.30682592e-04,  1.02153644e-02, -9.31500457e-04,
       -6.03267252e-02,  4.28329669e-02,  2.23814696e-02, -2.74674296e-02,
       -2.81390324e-02, -1.34990746e-02, -1.88973173e-02, -1.78640764e-02,
       -2.14541499e-02, -1.77538954e-02,  2.65041683e-02,  2.09890604e-05,
       -2.06341520e-02, -8.08069482e-03,  1.58628321e-03,  2.99699288e-02,
        2.19879672e-02,  5.02292961e-02, -3.25703025e-02, -1.95940338e-05,
       -5.34596182e-02, -2.14272812e-02, -3.04102257e-04, -8.49840324e-03,
       -3.55653698e-03, -2.86286250e-02, -4.96886931e-02, -1.47072719e-02,
        4.41317447e-02,  3.20917889e-02, -1.28480755e-02,  4.28970810e-03,
       -4.83294800e-02, -2.44745128e-02,  1.03392766e-03, -1.20740533e-02,
       -1.42332725e-02, -2.30670664e-02, -1.01170214e-02,  7.17609236e-03,
        1.70315132e-02, -

In [59]:
# 也可以推断一个句向量(未出现在语料中)
model.infer_vector(['硅谷动力', '消息', '国外', '媒体', '报道'])

array([ 0.00014774,  0.00332045,  0.0006638 ,  0.00094046, -0.00393822,
        0.00091024,  0.00201688,  0.00056447, -0.00936893, -0.00277192,
        0.00607381, -0.0014556 , -0.01528937,  0.00810291,  0.00352349,
       -0.01173889, -0.00494811, -0.00399501, -0.0073605 , -0.00900388,
       -0.00807428, -0.00563457,  0.00202666, -0.00394949, -0.00057615,
       -0.00294345,  0.00025875,  0.00512495,  0.00817535,  0.01458769,
       -0.01094608,  0.00055326, -0.01794113, -0.00264138,  0.00370824,
        0.00090517,  0.00375289, -0.01092687, -0.01581898, -0.00788368,
        0.01711205,  0.0092872 , -0.00408866,  0.00250408, -0.01427564,
       -0.00878029,  0.00445288, -0.002117  , -0.00219024, -0.00341219,
       -0.00307109,  0.0061007 ,  0.00063986, -0.00814158,  0.00081454,
        0.01061562, -0.01495458,  0.00306041, -0.00072068, -0.00011524,
        0.00461435, -0.00075104, -0.00763792,  0.00427098, -0.00050191,
        0.01248365,  0.00549311, -0.01308827, -0.00289718,  0.00

In [60]:
model[u'电脑']

array([ 0.05759174, -0.0324204 , -0.13748378, -0.10478242, -0.03704946,
        0.15005212,  0.20158792, -0.03228185, -0.1987218 , -0.00191699,
        0.14130056, -0.04760848, -0.5741621 ,  0.41417077,  0.21564123,
       -0.2294051 , -0.28026375, -0.1167319 , -0.16456772, -0.14838932,
       -0.20656772, -0.18901998,  0.22217557,  0.00584121, -0.1654536 ,
       -0.04413658,  0.01562118,  0.29066145,  0.1725013 ,  0.5451136 ,
       -0.30418026, -0.04748738, -0.51059467, -0.16401653,  0.0225423 ,
       -0.05175005, -0.02708316, -0.31013724, -0.47238162, -0.15122662,
        0.4072532 ,  0.281237  , -0.10308406,  0.01279924, -0.46868768,
       -0.27410498, -0.01236954, -0.15771626, -0.11562809, -0.2617204 ,
       -0.10056328,  0.07169583,  0.12291231, -0.26450318, -0.10790995,
        0.31910384, -0.41646147,  0.04331794,  0.13756318, -0.06318691,
        0.19886343, -0.11879745, -0.21870501,  0.18406904,  0.0334943 ,
        0.296223  ,  0.19541647, -0.41524127, -0.09041841,  0.19


## 6.案例实践
#### 电商产品评论数据情感分析

* 随着网上购物越来越流行，人们对于网上购物的需求变得越来越高，这让京东、淘宝等电商平台得到了很大的发展机遇。在这种电商平台激烈竞争的大背景下，除了提高商品质量、压低商品价格外，了解更多消费者的心声对于电商平台来说也变得越来越有必要，其中非常重要的方式就是对消费者的文本评论数据进行内在信息的数据挖掘分析，有利于对应商品的生产厂家自身竞争力的提升。
*  针对京东商城上“美的”品牌的热水器的消费者的文本评论数据，在对文本进行基本的机器预处理、中文分词、停用词过滤后，通过建立包括栈式自编码深度学习、语义网络与LDA主题模型等多种数据挖掘模型，实现对文本评论数据的倾向性判断以及所隐藏的信息挖掘并分析，以期望得到有价值的内在内容。

### 数据文件
* 京东用户评论数据

### 本节任务

对京东平台上的热水器评论进行文本挖掘分析
* 分析某一品牌热水器的用户情感倾向
* 从评论文本中挖掘出该品牌热水器的优点与不足

In [61]:
import jieba
import pandas as pd
from gensim import corpora, models

ModuleNotFoundError: No module named 'jieba'

### 1.提取评论数据

将品牌为“美的”的评论一列抽取，另存为meidi_jd.txt，编码为UTF-8

In [ ]:
inputfile = "data/huizong.csv"
outputfile = "data/meidi_jd.txt"
data = pd.read_csv(inputfile, encoding="utf-8")
data = data[[u"评论"]][data[u"品牌"] == u"美的"]
data.to_csv(outputfile, index=False, header=False, encoding="utf8")

In [ ]:
data

### 2.评论预处理
* 取到文本后，首先要进行文本评论数据的预处理。文本评论数据里存在大量价值含量很低甚至没有价值含量的条目，如果将这些评论数据也引入进行分词、词频统计乃至情感分析等，必然会对分析造成很大的影响，得到的结果的质量也必然是存在问题的。那么，在利用这些文本评论数据之前就必须先进行文本预处理，把大量的此类无价值含量的评论去除。
* 文本评论数据的预处理主要由3个部分组成：文本去重、机械压缩去词以及短句删除。

#### 删除重复评论

In [ ]:
inputfile = "data/meidi_jd.txt"
outputfile = "data/meidi_jd_process_1.txt"
data = pd.read_csv(inputfile, encoding="utf8", header=None)
l1 = len(data)

data = pd.DataFrame(data[0].unique())
l2 = len(data)
data.to_csv(outputfile, index=False, header=False, encoding="utf8")
print(u"删除了%s条评论" % (l1 - l2))

#### 删除前缀评分
利用正则去除一些数据

In [ ]:
inputfile1 = u"data/meidi_jd_process_end_负面情感结果.txt"
inputfile2 = u"data/meidi_jd_process_end_正面情感结果.txt"
outputfile1 = "data/meidi_jd_neg.txt"
outputfile2 = "data/meidi_jd_pos.txt"

data1 = pd.read_csv(inputfile1, encoding="utf8", header=None)
data2 = pd.read_csv(inputfile2, encoding="utf8", header=None)
data1.head()

In [ ]:
data1 = pd.DataFrame(data1[0].str.replace(".*?\d+?\\t ", ""))
data2 = pd.DataFrame(data2[0].str.replace(".*?\d+?\\t ", ""))

data1.to_csv(outputfile1, index=False, header=False, encoding="utf8")
data2.to_csv(outputfile2, index=False, header=False, encoding="utf8")
data1.head()

#### 分词
使用jieba分词

In [ ]:
inputfile1 = "data/meidi_jd_neg.txt"
inputfile2 = "data/meidi_jd_pos.txt"
outputfile1 = "data/meidi_jd_neg_cut.txt"
outputfile2 = "data/meidi_jd_pos_cut.txt"

data1 = pd.read_csv(inputfile1, encoding="utf8", header=None)
data2 = pd.read_csv(inputfile2, encoding="utf8", header=None)

def mycut(s): return " ".join(jieba.cut(s))

data1 = data1[0].apply(mycut)
data2 = data2[0].apply(mycut)

data1.to_csv(outputfile1, index=False, header=False, encoding="utf8")
data2.to_csv(outputfile2, index=False, header=False, encoding="utf8")

### 3.评论数据分析

#### LDA
分词之后的语义分析，LDA模型分析正面负面情感

In [ ]:
negfile = "data/meidi_jd_neg_cut.txt"
posfile = "data/meidi_jd_pos_cut.txt"
stoplist = "data/stoplist.txt"

neg = pd.read_csv(negfile, encoding="utf8", header=None)
pos = pd.read_csv(posfile, encoding="utf8", header=None)
"""
sep设置分割词，由于csv默认半角逗号为分割词，而且该词恰好位于停用词表中
所以会导致读取错误
解决办法是手动设置一个不存在的分割词，这里使用的是tipdm
参数engine加上，指定引擎，避免警告
"""
stop = pd.read_csv(stoplist, encoding="utf8", header=None, sep="tipdm", engine="python")

# pandas自动过滤了空格，这里手动添加
stop = [" ", ""] + list(stop[0])

# 定义分割函数，然后用apply进行广播， 去掉停用词
neg[1] = neg[0].apply(lambda s: s.split(" "))
neg[2] = neg[1].apply(lambda x: [i for i in x if i not in stop])
pos[1] = pos[0].apply(lambda s: s.split(" "))
pos[2] = pos[1].apply(lambda x: [i for i in x if i not in stop])

neg

In [ ]:
# 负面主题分析
# 建立词典
neg_dict = corpora.Dictionary(neg[2]) #词袋模型，将每个单词编码为整数
# 建立语料库
neg_corpus = [neg_dict.doc2bow(i) for i in neg[2]]  # bag of word 词袋模型编码
# LDA模型训练
neg_lda = models.LdaModel(neg_corpus, num_topics=3, id2word=neg_dict)

for i in range(3):
    print(neg_lda.print_topic(i))  #输出一个主题

In [ ]:
# 正面主题分析
# 以下同上
pos_dict = corpora.Dictionary(pos[2])
pos_corpus = [pos_dict.doc2bow(i) for i in pos[2]]
pos_lda = models.LdaModel(pos_corpus, num_topics=3, id2word=pos_dict)

for i in range(3):
    print(pos_lda.print_topic(i))

# Any Questions?